# 목적지향 대화시스템 NLU 시스템

### word2vec
- 보통 딥러닝은 입력층과 출력층 사이에 layer가 충분히 쌓여야 하는데 이건 하나의 입력층만 있어서 딥러닝이라고 하기 뭐함
- 그리고 그 입력층에는 활성화 함수가 없어서 projection layer라고도 불림
- 두 가지 모델 있음
  - CBOW : W(t)라는 중심 단어를 주변 단어들을 통해 예측
  - Skip-gram : W(t)라는 중심 단어를 통해 주변 단어들을 예측
  - 많은 논문에서는 skip-gram 성능이 더 좋다고 함

In [ ]:
pip install gensim==3.4.0

In [ ]:
pip install sentencepiece

In [ ]:
pip install pytorch-crf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive_project_root = 'drive/MyDrive/Colab Notebooks/'

In [ ]:
import os
import sys
import json
import torch
import random

import numpy as np
import pandas as pd

# conda install pytorch -c pytorch
# pip install torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from tqdm import tqdm
from tqdm import trange

# conda install gensim==3.4.0
# pip install --upgrade gensim==3.4.0
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

#pip install sentencepiece
#pip install pytorch-crf
from src.dataset import Preprocessing
from src.model import EpochLogger, MakeEmbed, save

'''
pip install pytorch-crf
https://pytorch-crf.readthedocs.io/en/stable/
'''
from torchcrf import CRF

from torch.autograd import Variable 


# 1. Embedding
### 1.1 Data Processing
- Intent Dataset을 corpus로 활용하여 Word2vec 학습을 위한 데이터 처리

In [ ]:
class Preprocessing:

    '''
    데이터의 최대 token길이가 10이지만
    실제 환경에서는 얼마의 길이가 들어올지 몰라 적당한 길이 부여
    '''
    
    def __init__(self, max_len = 20):
        self.max_len = max_len
        self.PAD = 0
    
    def pad_idx_sequencing(self, q_vec):
        q_len = len(q_vec)
        diff_len = q_len - self.max_len
        if(diff_len>0):
            q_vec = q_vec[:self.max_len]
            q_len = self.max_len
        else:
            pad_vac = [0] * abs(diff_len)
            q_vec += pad_vac

        return q_vec

class MakeDataset:
    def __init__(self):
        
        self.intent_data_dir = drive_project_root+"data/dataset/intent_data.csv"
        self.prep = Preprocessing()
    
    def tokenize(self, sentence):
        ''' 띄어쓰기 단위로 tokenize 적용'''
        return sentence.split()
    
    def tokenize_dataset(self, dataset):
        ''' Dataset에 tokenize 적용'''
        token_dataset = []
        for data in dataset:
            token_dataset.append(self.tokenize(data))
        return token_dataset
    
    def make_embed_dataset(self, ood = False):
        embed_dataset = pd.read_csv(self.intent_data_dir)
        embed_dataset = embed_dataset["question"].to_list()
        embed_dataset = self.tokenize_dataset(embed_dataset)

        return embed_dataset         

In [ ]:
dataset = MakeDataset()
embed_dataset = dataset.make_embed_dataset()
# embed_dataset

### 1.2 Embedding

In [ ]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    '''https://radimrehurek.com/gensim/models/callbacks.html'''
    '''학습 중간에 프린트를 하기 위한 logger'''
    def __init__(self):
        self.epoch = 0
        
    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

class MakeEmbed:
    '''https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec'''
    '''https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#online-training-resuming-training'''
    def __init__(self):
        self.model_dir = "./"
        self.vector_size = 300 # 임베딩 사이즈
        self.window_size = 3   # 몇개의 단어로 예측을 할것인지
        self.workers = 8       # 학습 스레드의 수
        self.min_count = 2     # 단어의 최소 빈도수 (해당 수 미만은 버려진다) -> 너무 적은 수의 단어는 정보가 약하고 학습이 잘 안 돼서
        self.iter = 1000       # 1epoch당 학습 수
        self.sg = 1            # 1: skip-gram, 0: CBOW
        self.model_file = drive_project_root+"/data/pretraining/word2vec_skipgram_{}_{}_{}".format(self.vector_size, self.window_size, self.min_count)
        self.epoch_logger = EpochLogger()  # 시작, 끝을 알려주기 위한 callback 함수

    def word2vec_init(self): # word2vec 초기화 및 세팅
        self.word2vec = Word2Vec(size=self.vector_size,
                         window=self.window_size,
                         workers=self.workers,
                         min_count=self.min_count,
                         compute_loss=True,
                         iter=self.iter)

    def word2vec_build_vocab(self, dataset): # 단어장 만들기
        self.word2vec.build_vocab(dataset)
        
    def word2vec_most_similar(self, query): # 비슷한 단어 계산
        print(self.word2vec.most_similar(query))
        
    def word2vec_train(self,embed_dataset, epoch = 0): # 학습
        if(epoch == 0):
            epoch = self.word2vec.epochs + 1
        self.word2vec.train(
            sentences=embed_dataset,
            total_examples=self.word2vec.corpus_count,
            epochs=epoch,
            callbacks=[self.epoch_logger]
        )

        self.word2vec.save(self.model_file + '.gensim')
        self.vocab = self.word2vec.wv.index2word
        self.vocab = {word: i for i, word in enumerate(self.vocab)}
        
    def load_word2vec(self):

        # 지정해 놓은 위치에 파일이 없는 경우
        if not os.path.exists(self.model_file+'.gensim'):
            raise Exception("모델 로딩 실패 "+ self.model_file+'.gensim')

        # 파일이 있는 경우 model load하고 vocab 생성
        # word2vec에 학습 데이터(vocab)에 없는 단어가 존재하면 error 발생함
        # 이를 막아주기 위해 <UNK> : unknown token insert -> index 1
        # padding을 할 거기 때문에 pad token도 추가 -> index 0
        # 최종 결과를 vocab에 저장
        self.word2vec = Word2Vec.load(self.model_file+'.gensim')
        self.vocab = self.word2vec.wv.index2word
        self.vocab.insert(0,"<UNK>") # vocab애 없는 토큰 등장할 경우를 대비한 <UNK> 토큰을 vocab에 삽입, index 1
        self.vocab.insert(0,"<PAD>") # 길이를 맞추기 위한 padding을 위해 <PAD> 토큰을 vacab에 삽입, index 0
        self.vocab = {word: i for i, word in enumerate(self.vocab)}
        
    def query2idx(self, query):
        sent_idx = []

        for word in query:
            if(self.vocab.get(word)):
                idx = self.vocab[word]
            else:
                idx = 1

            sent_idx.append(idx)

        return sent_idx

In [ ]:
embed = MakeEmbed()

embed.word2vec_init()

embed.word2vec.build_vocab(embed_dataset)

- 학습이 되지 않은 word2vec에게 '미세먼지'와 가장 가까운 단어가 무엇인지 질문
- random하게 나옴

In [ ]:
embed.word2vec.wv.most_similar("미세먼지")

- 10번의 epoch 학습
- 학습 후 word2vec에게 '미세먼지'와 가장 가까운 단어가 무엇인지 질문 : 먼지, 날씨, 모레 등

In [ ]:
embed.word2vec_train(embed_dataset,10)

In [ ]:
embed.word2vec.wv.most_similar("미세먼지")

### 1.3 word2index

In [ ]:
# 문장 하나 가져오기
sentence = embed_dataset[0]
sentence   # token화 됨

query2idx 함수
- query가 list이기 때문에 for문을 돌면서 단어 하나씩을 가져오고 그게 우리 단어장 안에 있는지
- 있으면 해당 index를 가져오고 없으면 index를 1로 출력

In [ ]:
embed.query2idx(sentence)

In [ ]:
w2i = embed.query2idx(sentence)

padding 추가
- 보통 NLP에서는 길이 제한을 위해 사용
- 장점 : 메모리도 아끼고 정보를 얼만큼만 받을지 사전에 설정
- 방법 : 문장 길이가 padding보다 적으면 0으로 채우고, 길면 padding만큼만으로 자름

In [ ]:
class Preprocessing:
    '''
    데이터의 최대 token길이가 10이지만
    실제 환경에서는 얼마의 길이가 들어올지 몰라 적당한 길이 부여
    '''
    
    def __init__(self, max_len = 20):
        self.max_len = max_len
        self.PAD = 0
    
    def pad_idx_sequencing(self, q_vec):
        q_len = len(q_vec)
        diff_len = q_len - self.max_len
        if(diff_len>0):
            q_vec = q_vec[:self.max_len]
            q_len = self.max_len
        else:
            pad_vac = [0] * abs(diff_len)
            q_vec += pad_vac

        return q_vec

In [ ]:
dataset.prep.pad_idx_sequencing(w2i)

### 1.4 word2vec load

- load 전 index 보면 '미세먼지'가 0번, '추천해'가 1번

In [ ]:
list(embed.vocab)[:5]

In [ ]:
embed.load_word2vec()

In [ ]:
len(embed.vocab)  # vocab 개수 : 1481

- load 후 PAD, UNK 추가됨
- index가 하나씩 밀리면서 '미세먼지'는 2번, '추천해'는 3번이 됨

In [ ]:
embed.vocab

---
# 2. Intent Classification (의도 분류)

### 2.1 Data Processing

In [ ]:
class MakeDataset:
    def __init__(self):
        
        self.intent_label_dir = "./data/dataset/intent_label.json"
        self.intent_data_dir = "./data/dataset/intent_data.csv"
        
        self.intent_label = self.load_intent_label()
        self.prep = Preprocessing()
    
    def load_intent_label(self):
        ''' 미리 만들어 둔 예측해야할 intent label 로드'''
        f = open(self.intent_label_dir, encoding="UTF-8") 
        intent_label = json.loads(f.read())
        self.intents = list(intent_label.keys())
        return intent_label
    
    def tokenize(self, sentence):
        ''' 띄어쓰기 단위로 tokenize 적용'''
        return sentence.split()
    
    def tokenize_dataset(self, dataset):
        ''' Dataset에 tokenize 적용'''
        token_dataset = []
        for data in dataset:
            token_dataset.append(self.tokenize(data))
        return token_dataset

    def make_intent_dataset(self, embed):
        ''' intent 분류를 위한 Dataset 생성'''
        intent_dataset = pd.read_csv(self.intent_data_dir) # 데이터 로딩

        labels = [self.intent_label[label] for label in intent_dataset["label"].to_list()] # label -> index
            
        intent_querys = self.tokenize_dataset(intent_dataset["question"].tolist()) # 사용자 발화 -> tokenize (embedding과 동일)
        
        dataset = list(zip(intent_querys, labels)) # (사용자 발화, intent) 형태로 가공
        intent_train_dataset, intent_test_dataset = self.word2idx_dataset(dataset, embed) # word2index
        return intent_train_dataset, intent_test_dataset
    
    def word2idx_dataset(self, dataset ,embed, train_ratio = 0.8):
        embed_dataset = []
        question_list, label_list = [], []
        flag = True
        random.shuffle(dataset) #  훈련용과 검증용으로 나눌때 intent 편형이 나타나지 않도록 데이터 셔플
        for query, label in dataset :
            q_vec = embed.query2idx(query) # 사용자 발화 index화
            q_vec = self.prep.pad_idx_sequencing(q_vec) # 사용자 발화 최대길이까지 padding

            question_list.append(torch.tensor([q_vec]))
            label_list.append(torch.tensor([label]))

        x = torch.cat(question_list)
        y = torch.cat(label_list)

        # 학습용과 검증용으로 나누기
        x_len = x.size()[0]
        y_len = y.size()[0]
        if(x_len == y_len):
            train_size = int(x_len*train_ratio)
            
            train_x = x[:train_size]
            train_y = y[:train_size]

            test_x = x[train_size+1:]
            test_y = y[train_size+1:]
            
            # TensorDataset으로 감싸기
            '''
             PyTorch의 TensorDataset은 tensor를 감싸는 Dataset입니다.

             인덱싱 방식과 길이를 정의함으로써 이것은 tensor의 첫 번째 차원을 따라 반복, 인덱스, 슬라이스를 위한 방법을 제공합니다.

             훈련할 때 동일한 라인에서 독립 변수와 종속 변수에 쉽게 접근할 수 있습니다.
            '''
            train_dataset = TensorDataset(train_x,train_y)
            test_dataset = TensorDataset(test_x,test_y)
            
            return train_dataset, test_dataset
            
        else:
            print("ERROR x!=y")
            

In [ ]:
dataset = MakeDataset()
intent_dataset = pd.read_csv(dataset.intent_data_dir)
intent_dataset.head()

In [ ]:
intent_dataset.groupby(['label']).count() 

In [ ]:
embed = MakeEmbed()
embed.load_word2vec()

batch_size = 128

intent_train_dataset, intent_test_dataset = dataset.make_intent_dataset(embed)

# 한번의 iter당 Batch size의 x, y를 제공한다.
train_dataloader = DataLoader(intent_train_dataset, batch_size=batch_size, shuffle=True) 

test_dataloader = DataLoader(intent_test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
intent_train_dataset.tensors

### 2.2 Modeling : Convolutional Neural Networks for Sentence Classification
- Yoon Kim, New York University
- tensorflow code : https://github.com/SeonbeomKim/TensorFlow-TextCNN/blob/master/TextCNN.py

In [ ]:
class textCNN(nn.Module):
    
    def __init__(self, w2v, dim, kernels, dropout, num_class):
        super(textCNN, self).__init__()
        # Word2vec으로 미리 학습해둔 임베딩 적용
        vocab_size = w2v.size()[0]
        emb_dim = w2v.size()[1]
        self.embed = nn.Embedding(vocab_size+2, emb_dim) # +2 : <UNK>, <PAD> -> 학습을 미리 하지 않았기 때문에 이 부분만 copy하기 위함
        self.embed.weight[2:].data.copy_(w2v)
        # self.embed.weight.requires_grad = False # 임베딩 레이어 학습 유무
        
        # 윈도우 사이즈가 다른 각각의 conv layer 를 nn.ModuleList로 저장
        # nn.Conv2d(in_channels, out_channels, kernel_size)
        self.convs = nn.ModuleList([nn.Conv2d(1, dim, (w, emb_dim)) for w in kernels])
        #Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        #FC layer
        self.fc = nn.Linear(len(kernels)*dim, num_class)
        
    def forward(self, x):
        emb_x = self.embed(x)
        emb_x = emb_x.unsqueeze(1)

        con_x = [conv(emb_x) for conv in self.convs] # 각 사이즈 별 결과를 list로 저장, 
        #[(out_channels, conv결과 길이),...]

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x] # 각 사이즈별 max_pool 결과 저장
        #[(256,1),...]

        fc_x = torch.cat(pool_x, dim=1) # concat하여 fc layer의 입력 형태로 만듬
        #(768,1) : 256 * 3 = 768

        fc_x = fc_x.squeeze(-1) # 차원 맞추기
        #(768)
        fc_x = self.dropout(fc_x)
        logit = self.fc(fc_x)
        return logit

# 모델의 가중치 저장을 위한 코드
def save(model, save_dir, save_prefix, epoch):
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    save_prefix = os.path.join(save_dir, save_prefix)
    save_path = '{}_steps_{}.pt'.format(save_prefix, epoch)
    torch.save(model.state_dict(), save_path)

In [ ]:
weights = embed.word2vec.wv.vectors # word2vec weight
weights = torch.FloatTensor(weights)

num_class = len(dataset.intent_label) 
model = textCNN(weights, 256, [3,4,5], 0.5, num_class)  # [3,4,5]: convolution, 0.5: Dropout, num_class: 4 (intent 총 개수)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
model

### 2.3 Training

In [ ]:
%%time
epoch = 10
prev_acc = 0
save_dir = "./data/pretraining/1_intent_clsf_model/"
save_prefix = "intent_clsf"
for i in range(epoch):
    steps = 0
    model.train() 
    #for data in train_dataloader:
    with tqdm(train_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {i}")
            x = data[0]
            target = data[1]
            logit = model.forward(x)
            
            optimizer.zero_grad()
            loss = F.cross_entropy(logit, target) # loass function
            loss.backward()
            optimizer.step()

            corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
            accuracy = 100.0 * corrects/x.size()[0]
            tepoch.set_postfix(loss=loss.item(), accuracy= accuracy.numpy())
            
    model.eval() # weight 업데이트 금지
    steps = 0
    accuarcy_list = []
    #for data in test_dataloader:
    with tqdm(test_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {i}")
            x = data[0]
            target = data[1]

            logit = model.forward(x)
            loss = F.cross_entropy(logit, target)
            corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
            accuracy = 100.0 * corrects/x.size()[0]
            accuarcy_list.append(accuracy.tolist())
            
            tepoch.set_postfix(loss=loss.item(), accuracy= sum(accuarcy_list)/len(accuarcy_list))
    
    # epoch 당 검증 셋의 정확도를 계산하고 이전 정확도 보다 높으면 저장     
    acc = sum(accuarcy_list)/len(accuarcy_list)
    if(acc>prev_acc):
        prev_acc = acc
        save(model, save_dir, save_prefix+"_"+str(round(acc,3)), i)

### 2.3 Load & Test

In [ ]:
model.load_state_dict(torch.load("./data/pretraining/save/1_intent_clsf_model/intent_clsf_97.217_steps_33.pt"))

model.eval()

In [ ]:
%%time
q = "제주도 오늘 날씨 알려줘"

x = dataset.prep.pad_idx_sequencing(embed.query2idx(dataset.tokenize(q)))

x = torch.tensor(x)
f = model(x.unsqueeze(0))

intent = dataset.intents[torch.argmax(f).tolist()]

print("발화 : " + q)
print("의도 : " + intent)

---
# 3. Entity Recognition

### 3.1 Data Processing

In [ ]:
'''
 PyTorch의 TensorDataset은 기본적으로 x[index], y[index]를 제공합니다.
 그 외에 추가로 제공하고 싶은게(문장 길이) 있으면 아래와 같이 커스텀이 가능합니다.
 여기서는 입력되는 문장의 길이를 제공 받아야해서 아래와 같이 커스텀을 하였습니다.
'''
class EntityDataset(data.Dataset):
    def __init__(self, x_tensor, y_tensor, lengths):
        super(EntityDataset, self).__init__()

        self.x = x_tensor
        self.y = y_tensor
        self.lengths = lengths
        
    def __getitem__(self, index):
        return self.x[index], self.y[index], self.lengths[index]

    def __len__(self):
        return len(self.x)
    
class MakeDataset:
    def __init__(self):
        
        self.entity_label_dir = drive_project_root+"data/dataset/entity_label.json"
        self.entity_data_dir = drive_project_root+"data/dataset/entity_data.csv"
        
        self.entity_label = self.load_entity_label()
        self.prep = Preprocessing()
        
    def load_entity_label(self):
        f = open(self.entity_label_dir, encoding="UTF-8")
        entity_label = json.loads(f.read())
        self.entitys = list(entity_label.keys())
        return entity_label
    
    
    def tokenize(self, sentence):
        return sentence.split()
    
    def tokenize_dataset(self, dataset):
        token_dataset = []
        for data in dataset:
            token_dataset.append(self.tokenize(data))
        return token_dataset
    
    def make_entity_dataset(self, embed):
        entity_dataset = pd.read_csv(self.entity_data_dir)
        entity_querys = self.tokenize_dataset(entity_dataset["question"].tolist())
        labels = []
        for label in entity_dataset["label"].to_list():
            temp = []
            for entity in label.split():
                temp.append(self.entity_label[entity])
            labels.append(temp)
        dataset = list(zip(entity_querys, labels))
        entity_train_dataset, entity_test_dataset = self.word2idx_dataset(dataset, embed)
        return entity_train_dataset, entity_test_dataset
    
    def word2idx_dataset(self, dataset ,embed, train_ratio = 0.8):
        embed_dataset = []
        question_list, label_list, lengths = [], [], []
        flag = True
        random.shuffle(dataset)
        
        # 발화와 label 모두 padding 필요
        # 발화와 label 길이가 같아야 하기 때문
        for query, label in dataset :
            q_vec = embed.query2idx(query)
            lengths.append(len(q_vec))
            
            q_vec = self.prep.pad_idx_sequencing(q_vec)

            question_list.append(torch.tensor([q_vec]))

            label = self.prep.pad_idx_sequencing(label) # 
            label_list.append(label)
            flag = False


        x = torch.cat(question_list)
        y = torch.tensor(label_list)

        x_len = x.size()[0]
        y_len = y.size()[0]
        if(x_len == y_len):
            train_size = int(x_len*train_ratio)
            
            train_x = x[:train_size]
            train_y = y[:train_size]

            test_x = x[train_size+1:]
            test_y = y[train_size+1:]

            train_length = lengths[:train_size]
            test_length = lengths[train_size+1:]

            train_dataset = EntityDataset(train_x,train_y,train_length)
            test_dataset = EntityDataset(test_x,test_y,test_length)
            
            return train_dataset, test_dataset
            
        else:
            print("ERROR x!=y")

In [ ]:
dataset = MakeDataset()

In [ ]:
#Inside, Out, Begin, End, Single
#IO :  TAG라면 I 을 아니면 O 로 태그
#BIO : TAG의 길이가 2이상이면 첫 번째 단어는 B를 붙이고 그 뒤의 단어들은 I를 붙인다
#BIOES : BIO에서 단어의 길이가 3이상인 단어는 마지막 단어에 E를 붙인다. 그리고 단어의 길이가 1이라면, S를 붙인다
# S : 단독
# B : 복합의 시작 (단독 사용 불가)
# I : 복합의 중간 (단독 사용 불가)
# E : 복합의 끝  (단독 사용 불가)
# O : 의미 없음
dataset.entity_label

In [ ]:
entity_dataset = pd.read_csv(dataset.entity_data_dir)

entity_dataset.head()

- intent classification은 label이 balance 해야 했는데, 여기서는 한 단어의 tag가 무엇이냐가 중요해서 balance 필요 없음

In [ ]:
entity_dataset.groupby(['label']).count() 

### 3.2 Bidirectional LSTM-CRF Models for Sequence Tagging
- Zhiheng Huang,Wei Xu,Kai Yu
- tensorflow code : https://github.com/ngoquanghuy99/POS-Tagging-BiLSTM-CRF/blob/main/model.py
- tensorflow, keras code : https://github.com/floydhub/named-entity-recognition-template/blob/master/ner.ipynb

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, w2v, tag_to_ix, hidden_dim, batch_size):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = w2v.size()[1]
        self.hidden_dim = hidden_dim
        self.vocab_size =  w2v.size()[0]
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.batch_size = batch_size
        self.START_TAG = "<START_TAG>"
        self.STOP_TAG = "<STOP_TAG>"
        
        self.word_embeds = nn.Embedding(self.vocab_size+2, self.embedding_dim)
        self.word_embeds.weight[2:].data.copy_(w2v)
        #self.word_embeds.weight.requires_grad = False
        
        # LSTM 파라미터 정의
        # bidirectional : 양방향 LSTM
        # num_layers    : layer의 수
        # batch_first   : pytorch에서 LSTM 입력의 기본값은 (Length,batch,Hidden) 순서 이므로 (batch,Length,Hidden)로 바꿔주기 위함
        # nn.LSTM(input_size, hidden_size, batch_first, num_layers)
        # hidden_size = hidden_dim // 2 인 이유는 bidirectional = True 이기 떄문입니다.
        self.lstm = nn.LSTM(self.embedding_dim, hidden_dim // 2, batch_first=True, num_layers=1, bidirectional=True)
    
        # LSTM의 출력을 태그 공간으로 대응시킵니다.
        # LSTM 출력을 CRF 입력으로 받기 전에 fully connected neural network를 통과해야 함
        # CRF 입력이 <문장 길이, 출력 tag 수>이기 때문
        # = 출력 tag 공간으로 mapping 하기 위해 필요
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
    
        self.hidden = self.init_hidden()
        
        # 출력층의 규칙학습을 위한 CRF 세팅
        self.crf = CRF(self.tagset_size, batch_first=True)
        
    def init_hidden(self):#(h,c)
        return (torch.randn(2, self.batch_size, self.hidden_dim // 2),  # bidirectional이기 때문에 2
                torch.randn(2, self.batch_size, self.hidden_dim // 2))

    def forward(self, sentence):
        # Bi-LSTM으로부터 배출 점수를 얻습니다.
        self.batch_size = sentence.size()[0]
        self.hidden = self.init_hidden()
        #(2,128,128),(2,128,128)
        embeds = self.word_embeds(sentence)
        #(128,20,300)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        #(128,20,256),((2,128,128),(2,128,128))
        lstm_feats = self.hidden2tag(lstm_out)#(batch, length,tagset_size)
        #(128,20,17)
        return lstm_feats
    
    def decode(self, logits, mask):
        """
        Viterbi Decoding의 구현체입니다.
        CRF 레이어의 출력을 prediction으로 변형합니다.
        :param logits: 모델의 출력 (로짓)
        :param mask: 마스킹 벡터
        :return: 모델의 예측 (prediction)
        
        각 단어의 자리마다
          word 1의 태그 확률        |  word2의 태그 확률
         'O': 확률0,              | 'O': 확률A,
         'B-DATE': 확률1,         | 'B-DATE': 확률B
         'B-LOCATION': 확률2,     | 'B-LOCATION': 확률C,
         'B-PLACE': 확률3,        | 'B-PLACE': 확률D,  
         'B-RESTAURANT': 확률4,   | 'B-RESTAURANT': 확률E,  
         'E-DATE': 확률5,         | 'E-DATE': 확률F,   
         'E-LOCATION': 확률6,     | 'E-LOCATION': 확률G, 
         'E-PLACE': 확률7,        | 'E-PLACE': 확률H,  
         'E-RESTAURANT': 확률8,   | 'E-RESTAURANT': 확률I, 
         'I-DATE': 확률9,         | 'I-DATE': 확률J,    
         'I-RESTAURANT': 확률10,  | 'I-RESTAURANT': 확률K,
         'S-DATE': 확률11,        | 'S-DATE': 확률L,      
         'S-LOCATION': 확률12,    | 'S-LOCATION': 확률M,
         'S-PLACE': 확률13,       | 'S-PLACE': 확률N,  
         'S-RESTAURANT': 확률14,  | 'S-RESTAURANT': 확률O,
         '<START_TAG>': 확률15,   | '<START_TAG>': 확률P, 
         '<STOP_TAG>': 확률15,    | '<STOP_TAG>': 확률Q,
         
         각각의 높은 확률을 뽑는 것은 보통의 딥러닝 방식으로 B단독이나 I단독, E단독같은 문제를 야기할 수 있습니다.
         태그들의 확률 값을 받아서 
         CRF는 태그들의 의존성을 학습할수 있어서 태그 시퀀스의 확률이 가장 높은 확률을 가지는 예측 시퀀스를 선택한다.
         그래서 B단독이나 I단독, E단독과 같은 문제를 없애줍니다.
         예를 들어 B-DATE, O 와 같은걸 출력하지 않습니다. (CRF는 S-DATE, O 라고 출력합니다.)
        """

        return self.crf.decode(logits, mask)
    
    def compute_loss(self, label, logits, mask):
        """
        학습을 위한 total loss를 계산합니다.
        :param label: label
        :param logits: logits
        :param mask: mask vector
        :return: total loss
        """

        log_likelihood = self.crf(logits, label, mask=mask, reduction='mean')
        return - log_likelihood  # Negative log likelihood loss

In [ ]:
embed = MakeEmbed()
embed.load_word2vec()

entity_train_dataset, entity_test_dataset = dataset.make_entity_dataset(embed)

train_dataloader = DataLoader(entity_train_dataset, batch_size=128, shuffle=True)

test_dataloader = DataLoader(entity_test_dataset, batch_size=128, shuffle=True)

In [ ]:
entity_train_dataset.x

In [ ]:
entity_train_dataset.y

In [ ]:
weights = embed.word2vec.wv.vectors
weights = torch.FloatTensor(weights)

# entity_label : 우리가 tag할 entity 개수 (17개)
# 256 : hidden dimension
# 128 : batch
model = BiLSTM_CRF(weights, dataset.entity_label, 256, 128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()

### 3.3 Training
- 현재 코드에서는 accuracy는 나오지 않음
- loss가 줄어들고 있으면 학습이 잘 되고 있는 것

In [ ]:
epoch = 5
prev_acc = 0
save_dir = "./data/pretraining/1_entity_recog_model/"
save_prefix = "entity_recog"
for i in range(epoch):
    steps = 0
    model.train()
    #for data in train_dataloader:
    with tqdm(train_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {i}")
            x = data[0]
            y = data[1]
            length = data[2]
            
            logits = model.forward(x)
            # CRF 할 때 문장 길이를 맞추기 위해 넣었던 padingd token은 굳이 필요없으니 그 부분을 마스킹하기위한 코드
            # 우리는 length값이 존재하여 length값을 이용해서 마스크를 생성해도 가능
            # 하지만 코드 간략화를 위해 pytorch에 where 함수를 이용해 마스크 생성
            # torch.where 함수 설명 : https://runebook.dev/ko/docs/pytorch/generated/torch.where
            mask = torch.where(x > 0, torch.tensor([1.]), torch.tensor([0.])).type(torch.uint8)
            loss = model.compute_loss(y,logits,mask)
            
            loss.backward()
            optimizer.step()

            tepoch.set_postfix(loss=loss.item())
            
    model.eval()
    steps = 0
    accuarcy_list = []
    #for data in test_dataloader:
    with tqdm(test_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {i}")
            x = data[0]
            y = data[1]
            length = data[2]
            
            mask = torch.where(x > 0, torch.tensor([1.]), torch.tensor([0.])).type(torch.uint8)
            logits = model.forward(x)
            
            predicts = model.decode(logits,mask)
            # decode함수를 통해 정확도 계산
            corrects = []
            for target, leng, predict in zip(y, length, predicts):
                corrects.append(target[:leng].tolist() == predict)

            accuracy = 100.0 * sum(corrects)/len(corrects)
            accuarcy_list.append(accuracy)
            
            loss = model.compute_loss(y,logits,mask)
            tepoch.set_postfix(loss=loss.item(), accuracy= sum(accuarcy_list)/len(accuarcy_list))
            
        acc = sum(accuarcy_list)/len(accuarcy_list)
        if(acc>prev_acc):
            prev_acc = acc
            save(model, save_dir, save_prefix+"_"+str(round(acc,3)), i)

### 3.4 Load & Test

In [ ]:
model.load_state_dict(torch.load("./data/pretraining/save/1_entity_recog_model/entity_recog_97.192_steps_7.pt"))

model.eval()

In [ ]:
%%time
q = "이번 주 날씨"
x = dataset.prep.pad_idx_sequencing(embed.query2idx(dataset.tokenize(q)))

x = torch.tensor(x)
f = model(x.unsqueeze(0))

mask = torch.where(x > 0, torch.tensor([1.]), torch.tensor([0.])).type(torch.uint8)

predict = model.decode(f,mask.view(1,-1))

# S : 단독
# B : 복합의 시작
# I : 복합의 중간
# E : 복합의 끝
tag = [dataset.entitys[p] for p in predict[0]]
for i, j in zip(q.split(' '),tag):
    print("단어 : "+i+" , "+"태그 : "+j)

In [ ]:
%%time
q = "나 내일 제주도 여행 가는데 미세먼지 알려줘"
x = dataset.prep.pad_idx_sequencing(embed.query2idx(dataset.tokenize(q)))

x = torch.tensor(x)
f = model(x.unsqueeze(0))

mask = torch.where(x > 0, torch.tensor([1.]), torch.tensor([0.])).type(torch.uint8)

predict = model.decode(f,mask.view(1,-1))

# S : 단독
# B : 복합의 시작
# I : 복합의 중간
# E : 복합의 끝
tag = [dataset.entitys[p] for p in predict[0]]
for i, j in zip(q.split(' '),tag):
    print("단어 : "+i+" , "+"태그 : "+j)

---
# 4. OOD (Out of Domain) 분류

### Deep Averaging Network
- 단어 embedding의 평균을 내고 linear layer에 들어가서 문장 embedding 생성
- 정확도가 살짝 낮아도 모델이 가벼워서 학습 속도가 매우 빠름
  - Transformer는 layer가 많아서 계산량이 많다 보니 속도가 좀 느림


### 4.1 Data Processing

In [ ]:


class MakeDataset:
    def __init__(self):
        
        self.intent_ood_label_dir = "./data/dataset/intent_label_with_ood.json"
        self.intent_data_dir = "./data/dataset/intent_data.csv"
        self.ood_data_dir = "./data/dataset/ood_data.csv"
        
        self.intent_ood_label = self.load_intent_ood_label()
        self.prep = Preprocessing()
    
    def load_intent_ood_label(self):
        f = open(self.intent_ood_label_dir, encoding="UTF-8")
        intent_ood_label = json.loads(f.read())
        self.intents_ood = list(intent_ood_label.keys())
        return intent_ood_label
    
    def tokenize(self, sentence):
        return sentence.split()
    
    def tokenize_dataset(self, dataset):
        token_dataset = []
        for data in dataset:
            token_dataset.append(self.tokenize(data))
        return token_dataset

    def make_ood_dataset(self, embed):
        intent_dataset = pd.read_csv(self.intent_data_dir)
        ood_dataset = pd.read_csv(self.ood_data_dir)#.sample(frac=1).reset_index(drop=True)
        intent_dataset = pd.concat([intent_dataset,ood_dataset])
        labels = []
        for label in intent_dataset["label"].to_list():
            if(label == "OOD"):
                labels.append(0)
            else:
                labels.append(1)
            
        intent_querys = self.tokenize_dataset(intent_dataset["question"].tolist())
        
        dataset = list(zip(intent_querys, labels))
        intent_train_dataset, intent_test_dataset = self.word2idx_dataset(dataset, embed)
        return intent_train_dataset, intent_test_dataset
    
    
    def word2idx_dataset(self, dataset ,embed, train_ratio = 0.8):
        embed_dataset = []
        question_list, label_list = [], []
        flag = True
        random.shuffle(dataset)
        for query, label in dataset :
            q_vec = embed.query2idx(query)
            q_vec = self.prep.pad_idx_sequencing(q_vec)

            question_list.append(torch.tensor([q_vec]))

            label_list.append(torch.tensor([label]))

        x = torch.cat(question_list)
        y = torch.cat(label_list)

        x_len = x.size()[0]
        y_len = y.size()[0]
        if(x_len == y_len):
            train_size = int(x_len*train_ratio)
            
            train_x = x[:train_size]
            train_y = y[:train_size]

            test_x = x[train_size+1:]
            test_y = y[train_size+1:]
            
            train_dataset = TensorDataset(train_x,train_y)
            test_dataset = TensorDataset(test_x,test_y)
            
            return train_dataset, test_dataset
            
        else:
            print("ERROR x!=y")
            


In [ ]:
dataset = MakeDataset()

In [ ]:
pd.read_csv(dataset.ood_data_dir)

In [ ]:
pd.read_csv(dataset.intent_data_dir)

In [ ]:
embed = MakeEmbed()
embed.load_word2vec()

batch_size = 128

ood_train_dataset, ood_test_dataset = dataset.make_ood_dataset(embed)

train_dataloader = DataLoader(ood_train_dataset, batch_size=batch_size, shuffle=True)

test_dataloader = DataLoader(ood_test_dataset, batch_size=batch_size, shuffle=True)

### 4.2 Deep Unordered Composition Rivals Syntactic Methods for Text Classification
- Iyyer
- tensorflow code(tf-hub로 제공) : https://tfhub.dev/google/universal-sentence-encoder/4
- keras code : https://github.com/candlewill/Vecamend-master2/blob/master/dan.py#L41

In [ ]:
class DAN(nn.Module):
    
    def __init__(self, w2v, dim, dropout, num_class = 2):
        super(DAN, self).__init__()
        #load pretrained embedding in embedding layer.
        vocab_size = w2v.size()[0]
        emb_dim = w2v.size()[1]
        self.embed = nn.Embedding(vocab_size+2, emb_dim)
        self.embed.weight[2:].data.copy_(w2v)
        #self.embed.weight.requires_grad = False
        
        self.dropout1 = nn.Dropout(dropout)
        self.bn1 = nn.BatchNorm1d(emb_dim)
        self.fc1 = nn.Linear(emb_dim, dim)
        self.dropout2 = nn.Dropout(dropout)
        self.bn2 = nn.BatchNorm1d(dim)
        self.fc2 = nn.Linear(dim, num_class)
        
    def forward(self, x):
        emb_x = self.embed(x)
        #(128,20,300)
        x = emb_x.mean(dim=1)
        #(128,300)
        x = self.dropout1(x)
        x = self.bn1(x)
        x = self.fc1(x)
        #(128,256)
        x = self.dropout2(x)
        x = self.bn2(x)
        logit = self.fc2(x)
        #(128,2 )
        return logit

In [ ]:
weights = embed.word2vec.wv.vectors
weights = torch.FloatTensor(weights)

model = DAN(weights, 256, 0.5, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()

### 4.3 Training

In [ ]:
epoch = 5
prev_acc = 0
save_dir = drive_project_root+"/data/pretraining/1_ood_clsf_model//"
save_prefix = "ood_clsf"
for i in range(epoch):
    steps = 0
    model.train()
    #for data in train_dataloader:
    with tqdm(train_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {i}")
            x = data[0]
            target = data[1]
            logit = model.forward(x)
            optimizer.zero_grad()
            loss = F.cross_entropy(logit, target)
            loss.backward()
            optimizer.step()

            corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
            accuracy = 100.0 * corrects/x.size()[0]
            tepoch.set_postfix(loss=loss.item(), accuracy= accuracy.numpy())
            
    model.eval()
    steps = 0
    accuarcy_list = []
    # 현재 accuracy가 이전 accuracy보다 높으면 저장
    #for data in test_dataloader:
    with tqdm(test_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {i}")
            x = data[0]
            target = data[1]

            logit = model.forward(x)
            loss = F.cross_entropy(logit, target)
            corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
            accuracy = 100.0 * corrects/x.size()[0]
            accuarcy_list.append(accuracy.tolist())
            
            tepoch.set_postfix(loss=loss.item(), accuracy= sum(accuarcy_list)/len(accuarcy_list))
            
    acc = sum(accuarcy_list)/len(accuarcy_list)
    if(acc>prev_acc):
        prev_acc = acc
        save(model, save_dir, save_prefix+"_"+str(round(acc,3)), i)

### 4.4 Load & Test

In [ ]:
model.load_state_dict(torch.load("./data/pretraining/save/1_ood_clsf_model/ood_clsf_99.724_steps_5.pt"))

model.eval()

- %%time : 이 cell이 돌 때 걸리는 시간

In [ ]:
%%time
q = "제주도"

x = dataset.prep.pad_idx_sequencing(embed.query2idx(dataset.tokenize(q)))

x = torch.tensor(x)
f = model(x.unsqueeze(0))

y = torch.argmax(f).tolist()

print("발화 : " + q)
if(not y):
    print("ood")
else:
    print("intent")


In [ ]:
%%time
q = "제주도 날씨"

x = dataset.prep.pad_idx_sequencing(embed.query2idx(dataset.tokenize(q)))

x = torch.tensor(x)
f = model(x.unsqueeze(0))

y = torch.argmax(f).tolist()

print("발화 : " + q)
if(not y):
    print("ood")
else:
    print("intent")
